In [1]:
!pip install gensim

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from gensim import corpora, models
from typing import List
import string
import spacy
import ast
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


# Data Retrieval

In [9]:
columns_to_load =['trope_name', 'trope_description_partial_clean']
df = pd.read_csv('../data_clean/parsed_tropes_clean.csv',usecols=columns_to_load)

df.head()

,trope_name,trope_description_partial_clean
0,Aardvark Trunks,"['', '', ' ', '', ""Anteaters and aardvarks are..."
1,Abandoned Area,"['', '', ' Image from Pripyat, a ghost town ne..."
2,Abandoned Camp Ruins,"['', '', ' ', '', ""You're lost in the woods , ..."
3,Abandoned Catchphrase,"['', '', ' ', ""Character Catchphrases are a gr..."
4,Abandoned Hospital,"['', '', ' ', '', '', 'The creepy abandoned ho..."


# LDA Modeling

In [11]:
columns_to_load =['trope_name', 'trope_description_trad_clean']
df = pd.read_csv('../data_clean/parsed_tropes_clean.csv',usecols=columns_to_load, converters={
    'trope_description_trad_clean': ast.literal_eval})

df.head()

,trope_name,trope_description_trad_clean
0,Aardvark Trunks,"[anteater, aardvark, somewhat, common, fiction..."
1,Abandoned Area,"[image, pripyat, ghost, town, near, chernobyl,..."
2,Abandoned Camp Ruins,"[lose, wood, middle, night, creepy, sound, bra..."
3,Abandoned Catchphrase,"[character, catchphrase, great, simple, way, h..."
4,Abandoned Hospital,"[creepy, abandon, mental, institution, common,..."


In [15]:
!pip install pyLDAvis

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.6 MB 4.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.5/2.6 MB 16.2 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 20.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.6 MB 45.1 MB/s eta 0:00:01
   ------- -------------------------------- 2.3/11.6 MB 48.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.6 MB 46.5 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.6 MB 46.0 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.6 MB 45.8 MB/s eta 0:00:01
   ---------------------------------------  11.6/11.6 MB 46.7 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 43.7 MB/s eta 0:00:00
Using cached t

In [4]:
from gensim.models import CoherenceModel
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [25]:
!pip install --upgrade pandas pyLDAvis joblib

In [12]:
documents = df['trope_description_trad_clean']

In [13]:

dictionary = corpora.Dictionary(documents)
doc_term_matrix = [dictionary.doc2bow(text) for text in documents]

In [21]:
num_topics_range = range(5, 21)  # Adjust the range based on your requirements

# Initialize lists to store coherence scores and the best model
coherence_scores = []
best_model = None
best_score = -1

# Iterate over the range of candidate numbers of topics
for num_topics in num_topics_range:
    # Train the LDA model
    lda_model = models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=50)

    # Compute coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # Store the coherence score
    coherence_scores.append(coherence_score)
    print(f"Number of Topics: {num_topics}, Coherence Score: {coherence_score}")

    # Update the best model if this score is higher than the previous best
    if coherence_score > best_score:
        best_score = coherence_score
        best_model = lda_model

# Find the optimal number of topics with the highest coherence score
optimal_num_topics = num_topics_range[coherence_scores.index(max(coherence_scores))]

# Print the coherence scores for each number of topics
for num_topics, coherence_score in zip(num_topics_range, coherence_scores):
    print(f"Number of Topics: {num_topics}, Coherence Score: {coherence_score}")

# Print the optimal number of topics
print(f"\nOptimal Number of Topics: {optimal_num_topics}")

# Save the best model
best_model.save('best_lda_model.model')

print(f"The best LDA model has been saved as 'best_lda_model.model' with a coherence score of {best_score}.")

Number of Topics: 5, Coherence Score: 0.2881694616183096
Number of Topics: 6, Coherence Score: 0.34274968371065945
Number of Topics: 7, Coherence Score: 0.36478123456025
Number of Topics: 8, Coherence Score: 0.34099443787767436
Number of Topics: 9, Coherence Score: 0.3887366013115313
Number of Topics: 10, Coherence Score: 0.3670552331839626
Number of Topics: 11, Coherence Score: 0.40120474571393727
Number of Topics: 12, Coherence Score: 0.4064436008538534
Number of Topics: 13, Coherence Score: 0.40271193000817973
Number of Topics: 14, Coherence Score: 0.43178047670642805
Number of Topics: 15, Coherence Score: 0.4191294427837787
Number of Topics: 16, Coherence Score: 0.42848434537545793
Number of Topics: 17, Coherence Score: 0.4337368013192866
Number of Topics: 18, Coherence Score: 0.42577971295962486
Number of Topics: 19, Coherence Score: 0.4432608596457694
Number of Topics: 20, Coherence Score: 0.44641027340214806
Number of Topics: 5, Coherence Score: 0.2881694616183096
Number of Topi

In [14]:
lda_model = models.LdaModel.load('best_lda_model.model')

topics = lda_model.print_topics(num_topics=20, num_words=10)
for topic in topics:
    print(topic)

(0, '0.034*"human" + 0.013*"may" + 0.012*"alien" + 0.011*"trope" + 0.010*"example" + 0.010*"food" + 0.010*"like" + 0.010*"robot" + 0.009*"also" + 0.008*"body"')
(1, '0.024*"story" + 0.022*"time" + 0.021*"world" + 0.009*"plot" + 0.009*"work" + 0.008*"trope" + 0.008*"fiction" + 0.008*"one" + 0.008*"setting" + 0.007*"real"')
(2, '0.044*"child" + 0.027*"parent" + 0.021*"kid" + 0.020*"family" + 0.018*"old" + 0.014*"young" + 0.012*"may" + 0.010*"age" + 0.010*"adult" + 0.010*"one"')
(3, '0.042*"power" + 0.022*"magic" + 0.017*"may" + 0.016*"god" + 0.012*"use" + 0.011*"also" + 0.009*"trope" + 0.009*"see" + 0.009*"magical" + 0.008*"often"')
(4, '0.064*"game" + 0.022*"player" + 0.015*"video" + 0.010*"one" + 0.009*"level" + 0.008*"use" + 0.007*"time" + 0.007*"make" + 0.007*"play" + 0.007*"also"')
(5, '0.091*"animal" + 0.024*"creature" + 0.024*"dog" + 0.023*"monster" + 0.021*"pet" + 0.019*"cat" + 0.014*"bird" + 0.013*"like" + 0.012*"giant" + 0.012*"human"')
(6, '0.029*"woman" + 0.026*"girl" + 0.024

In [16]:
# Visualize the topics
vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary, n_jobs=1)
pyLDAvis.show(vis_data)

OSError: [Errno 22] Invalid argument: 'https://cdn.jsdelivr.net/gh/bmabey/pyLDAvis@3.4.0/pyLDAvis/js/ldavis.v1.0.0.css'